### Convert webpage to pdf

Method1: the simplest way

In [6]:
from weasyprint import HTML
HTML("https://news.sina.com.cn/gov/xlxw/2025-07-03/doc-infecyiv4278789.shtml").write_pdf("output3.pdf")

#### 版本8
增加新函数check_content_status，返回页面信息（字典）
完整版，较稳定，测试till 7.23
python venv：deep_learning

In [7]:
import asyncio
import time
from playwright.async_api import async_playwright

async def check_content_status(page):
    """检查页面内容加载状态（返回字典）"""
    try:
        return await robust_evaluate(page, """() => {
            // 获取所有图片
            const imgs = Array.from(document.querySelectorAll('img'));
            
            // 可见性检测函数
            const isVisible = (el) => {
                if (!el) return false;
                const rect = el.getBoundingClientRect();
                return rect.width > 0 && rect.height > 0;
            };
            
            // 返回结构化数据
            return {
                contentHeight: document.body.scrollHeight,
                contentReady: document.getElementById('js_content')?.offsetHeight > 2000,
                visibleImgs: imgs.filter(isVisible).length,
                loadedImgs: imgs.filter(img => img.complete && img.naturalHeight > 0).length,
                totalImgs: imgs.length
            };
        }""")
    except Exception as e:
        print(f"⚠️ 内容检查失败: {str(e)}")
        return None

async def robust_evaluate(page, expression, max_retries=3, delay=1):
    """带自动重试的evaluate函数（兼容异步）"""
    for attempt in range(max_retries):
        try:
            # 自动包装异步代码
            if "await" in expression and not expression.strip().startswith("async"):
                expression = f"async function() {{{expression}}}"
            return await page.evaluate(expression)
        except Exception as e:
            if attempt == max_retries - 1:
                raise
            print(f"🔄 评估失败，正在重试 ({attempt + 1}/{max_retries})")
            await asyncio.sleep(delay)

async def robust_operation(page, operation, *args, max_retries=3, delay=1, **kwargs):
    """增强版安全操作函数，支持任意页面方法"""
    for attempt in range(max_retries):
        try:
            if page.is_closed():
                raise RuntimeError("页面已关闭")
            return await getattr(page, operation)(*args, **kwargs)
        except Exception as e:
            if attempt == max_retries - 1:
                raise
            print(f"🔄 {operation}操作失败，正在重试 ({attempt + 1}/{max_retries})")
            await asyncio.sleep(delay)

async def save_wechat_to_pdf(url, output_path="output.pdf", max_attempts=3):
    """终极稳定版微信文章转PDF"""
    browser = None
    context = None
    page = None
    last_error = None
    
    for attempt in range(1, max_attempts + 1):
        print(f"\n=== 第 {attempt} 次尝试 ===")
        
        try:
            async with async_playwright() as p:
                # 1. 启动浏览器（新上下文隔离）
                browser = await p.chromium.launch(
                    headless=False,
                    slow_mo=800,  # 更长的操作间隔
                    timeout=150000,
                    channel="chrome",  # 使用稳定版Chrome
                    args=[
                        f"--window-size=1240,1754",  # A4比例(1240x1754像素 @150dpi)
                        "--force-device-scale-factor=1.0"
                    ]
                )
                context = await browser.new_context(
                    user_agent="Mozilla/5.0 (iPhone; CPU iPhone OS 15_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148 MicroMessenger/8.0.28(0x18001c2f)",
                    viewport={"width": 1240, "height": 1754},  # 对应210mm×297mm @150dpi
                    #viewport={"width": 375, "height": 812},  # 移动端尺寸
                    device_scale_factor=1.0,
                )
                page = await context.new_page()

                # 2. 设置请求头
                await robust_operation(page, "set_extra_http_headers", {
                    "Referer": "https://mp.weixin.qq.com/",
                    "Accept-Language": "zh-CN,zh;q=0.9"
                })

                # 强制设置页面缩放（关键修复）
                await robust_evaluate(page, """() => {
                    const meta = document.createElement('meta');
                    meta.name = 'viewport';
                    meta.content = 'width=device-width, initial-scale=1.0, maximum-scale=1.0';
                    document.head.appendChild(meta);
                    return true;  // 明确返回结果
                }""")

                # 页面样式预处理
                await robust_operation(page, "add_init_script", """() => {
                    // 提前注入样式防止布局抖动
                    const style = document.createElement('style');
                    style.textContent = `
                        body { margin:0 !important; padding:0 !important }
                        #js_content, .rich_media_content { 
                            width:100% !important; 
                            max-width:none !important 
                        }
                    `;
                    document.head.appendChild(style);
                }""")
            
                # 3. 智能加载页面
                print("🌐 正在加载页面...")
                await robust_operation(page, "goto", url,
                    wait_until="networkidle",
                    timeout=150000,
                    referer="https://mp.weixin.qq.com/"
                )

                # 4. 反爬检测与处理
                if await robust_evaluate(page, "() => document.title.includes('验证')"):
                    print("🛡️ 检测到验证页面，尝试自动处理...")
                    # 点击操作仍需使用 robust_operation（因为不是 evaluate 操作）
                    await robust_operation(page, "click", "text=继续阅读", timeout=5000)
                    await robust_operation(page, "wait_for_load_state", "networkidle")

                # 5. 内容预处理
                print("🛠️ 异步加载图片资源...")
                await robust_evaluate(page, """async () => {
                    // 1. 处理懒加载图片（异步等待每张图加载）
                    const imgs = document.querySelectorAll('img[data-src]');
                    for (const img of imgs) {
                        img.src = img.dataset.src;
                        await new Promise((resolve) => {
                            if (img.complete) resolve();
                            else img.onload = resolve;
                        });
                    }
                    
                    // 2. 原同步操作保持不变
                    document.querySelectorAll('[style*="background-image"]').forEach(el => {
                        const bg = window.getComputedStyle(el).backgroundImage;
                        if (bg.includes('url') && !bg.includes('loaded')) {
                            el.style.backgroundImage = bg.replace('loading', 'loaded');
                        }
                    });
                    
                    // 3. 移除干扰元素（同步）
                    document.querySelectorAll('.footer, .ad-banner').forEach(el => {
                        el.remove();
                    });
    
                    // 确保正文可见
                    const content = document.getElementById('js_content');
                    if (content) content.style.visibility = 'visible';
                    
                    return true;
                }""")

                # 6. 滚动加载完整内容
                print("📜 模拟滚动加载...")
                final_height = await robust_operation(page, "evaluate", """async () => {
                    let lastHeight = 0, stableCount = 0;
                    while (stableCount < 3) {  // 连续3次高度不变才停止
                        window.scrollTo(0, document.body.scrollHeight);
                        await new Promise(r => setTimeout(r, 2000));
                        const newHeight = document.body.scrollHeight;
                        if (Math.abs(newHeight - lastHeight) < 50) stableCount++;
                        else stableCount = 0;
                        lastHeight = newHeight;
                    }
                    return lastHeight;
                }""")

                #  重置页面样式（保证内容撑满宽度）
                ret = await robust_evaluate(page, """() => {
                    document.body.style.margin = '0';
                    document.body.style.padding = '0';
                    const container = document.getElementById('js_content') || document.body;
                    container.style.width = '100%';
                    container.style.maxWidth = 'none';
                    return true;  // 明确返回执行结果
                }""")

                # 在滚动加载后添加图片加载状态检查
                loaded_status = await robust_evaluate(page, """() => {
                    const imgs = Array.from(document.querySelectorAll('img'));
                    return {
                        total: imgs.length,
                        loaded: imgs.filter(img => img.complete && img.naturalHeight > 0).length
                    };
                }""")
                print(f"📊 图片加载状态: {loaded_status['loaded']}/{loaded_status['total']}")

                # 检查页面状态
                status = await check_content_status(page)
                if status or status['loadedImgs'] < status['totalImgs'] * 0.8:
                    print(f"📊 内容状态: 高度={status['contentHeight']}px | "
                          f"图片加载={status['loadedImgs']}/{status['totalImgs']}")
                    print("⚠️ 图片加载不完全，可能影响PDF质量")
                else:
                    print("❌ 内容检查失败，继续执行...")
                
                # 强制等待剩余图片（可选）
                #await asyncio.sleep(3000)
                
                # 8. 生成PDF（带智能分页）
                print("🖨️ 正在生成A4尺寸PDF...")
                await robust_operation(page, "pdf",
                    path=output_path,
                    format="A4",
                    width="210mm",          # A4标准宽度
                    height="297mm",         # A4标准高度
                    margin={
                        "top": "15mm",
                        "right": "15mm",
                        "bottom": "15mm",
                        "left": "15mm"
                    },
                    print_background=True, 
                    scale=0.95,              # 保证边距
                    # 关键参数：设置初始视图
                    display_header_footer=False,
                    prefer_css_page_size=False,  # 关键参数
                    # 强制从第一页开始（PDF打开参数）
                    tagged=True,  # 启用文档标签
                    outline=False  # 禁用大纲视图记忆位置
                )

                print(f"✅ 成功生成PDF: {output_path}")
                return True

        except Exception as e:
            last_error = str(e)
            print(f"⚠️ 尝试 {attempt} 失败: {last_error}")
            
            # 增强错误诊断
            timestamp = int(time.time())
            try:
                if page and not page.is_closed():
                    await robust_operation(page, "screenshot", 
                        path=f"error_{timestamp}.png",
                        full_page=True,
                        quality=90
                    )
                    html = await robust_operation(page, "content")
                    with open(f"debug_{timestamp}.html", "w", encoding="utf-8") as f:
                        f.write(html)
            except Exception as debug_error:
                print(f"⚠️ 调试信息保存失败: {debug_error}")

            if attempt < max_attempts:
                print(f"🔄 {max_attempts - attempt}次重试剩余...")
                await asyncio.sleep(3)  # 重试间隔

        finally:
            # 安全释放资源
            cleanup_tasks = []
            if page and not page.is_closed():
                cleanup_tasks.append(page.close())
            if context:
                cleanup_tasks.append(context.close())
            if browser:
                cleanup_tasks.append(browser.close())
            
            if cleanup_tasks:
                await asyncio.gather(*cleanup_tasks, return_exceptions=True)

    print(f"❌ 全部{max_attempts}次尝试失败，最后错误: {last_error}")
    return False


# 调用示例
url = "https://mp.weixin.qq.com/s?__biz=MzkzNDQyNjM2Ng==&mid=2247484776&idx=1&sn=6a73d5229fa01fb1b863c91d6c86c154&chksm=c2bc26f9f5cbafefacf11b49f45c10f168a7932168d6d309f597acc77b5fb09cdf1ac9874364&cur_album_id=3308113845693104131&scene=189#wechat_redirect"
success = await save_wechat_to_pdf(url)
print("执行结果:", "成功" if success else "失败")


=== 第 1 次尝试 ===
🌐 正在加载页面...
🛠️ 异步加载图片资源...
📜 模拟滚动加载...
📊 图片加载状态: 14/19
📊 内容状态: 高度=8937px | 图片加载=14/19
⚠️ 图片加载不完全，可能影响PDF质量
🖨️ 正在生成A4尺寸PDF...
✅ 成功生成PDF: output.pdf
执行结果: 成功


#### 版本8.1
使用PyPDF2 后处理水印，不太成功


In [ ]:
from PyPDF2 import PdfReader, PdfWriter
from PyPDF2.generic import RectangleObject, NameObject
from PyPDF2 import PageObject

def remove_watermark_by_coordinates(input_path, output_path):
    # 读取PDF文件
    reader = PdfReader(input_path)
    writer = PdfWriter()
    
    for page in reader.pages:
        # 创建空白覆盖页（修正括号问题）
        overlay = PageObject.create_blank_page(
            width=float(page.mediabox.width),
            height=float(page.mediabox.height)
        )
        
        # 定义水印区域（示例值，需根据实际情况调整）
        watermark_rect = RectangleObject([
            float(page.mediabox.right) - 100,  # 右边界向左100点
            float(page.mediabox.bottom),       # 底部
            float(page.mediabox.right),        # 右边界
            float(page.mediabox.bottom) + 50   # 向上50点
        ])
        
        # 合并原始页面和覆盖层
        overlay.merge_page(page)
        
        # 添加白色矩形覆盖水印
        if hasattr(overlay, 'add_rect'):
            overlay.add_rect(watermark_rect, fill=(1, 1, 1))  # RGB白色
        else:
            annots = overlay.get('/Annots', [])
            annots.append({
                NameObject('/Type'): NameObject('/Annot'),
                NameObject('/Subtype'): NameObject('/Square'),
                NameObject('/Rect'): watermark_rect,
                NameObject('/F'): 4,
                NameObject('/C'): [1, 1, 1],
                NameObject('/M'): 'Watermark removal'
            })
            overlay[NameObject('/Annots')] = annots
        
        writer.add_page(overlay)
    
    # 保存处理后的PDF
    with open(output_path, "wb") as f:
        writer.write(f)

# 使用方法
input_pdf = "output.pdf"
output_pdf = "output_clean.pdf"

remove_watermark_by_coordinates(input_pdf, output_pdf)
print(f"已处理完成：{output_pdf}")

#### 版本9
去除右下角水印，不太成功

In [81]:
import asyncio
import time
from playwright.async_api import async_playwright

async def check_content_status(page):
    """检查页面内容加载状态（返回字典）"""
    try:
        return await robust_evaluate(page, """() => {
            // 获取所有图片
            const imgs = Array.from(document.querySelectorAll('img'));
            
            // 可见性检测函数
            const isVisible = (el) => {
                if (!el) return false;
                const rect = el.getBoundingClientRect();
                return rect.width > 0 && rect.height > 0;
            };
            
            // 返回结构化数据
            return {
                contentHeight: document.body.scrollHeight,
                contentReady: document.getElementById('js_content')?.offsetHeight > 2000,
                visibleImgs: imgs.filter(isVisible).length,
                loadedImgs: imgs.filter(img => img.complete && img.naturalHeight > 0).length,
                totalImgs: imgs.length
            };
        }""")
    except Exception as e:
        print(f"⚠️ 内容检查失败: {str(e)}")
        return None

async def robust_evaluate(page, expression, max_retries=3, delay=1):
    """带自动重试的evaluate函数（兼容异步）"""
    for attempt in range(max_retries):
        try:
            # 自动包装异步代码
            if "await" in expression and not expression.strip().startswith("async"):
                expression = f"async function() {{{expression}}}"
            return await page.evaluate(expression)
        except Exception as e:
            if attempt == max_retries - 1:
                raise
            print(f"🔄 评估失败，正在重试 ({attempt + 1}/{max_retries})")
            await asyncio.sleep(delay)

async def robust_operation(page, operation, *args, max_retries=3, delay=1, **kwargs):
    """增强版安全操作函数，支持任意页面方法"""
    for attempt in range(max_retries):
        try:
            if page.is_closed():
                raise RuntimeError("页面已关闭")
            return await getattr(page, operation)(*args, **kwargs)
        except Exception as e:
            if attempt == max_retries - 1:
                raise
            print(f"🔄 {operation}操作失败，正在重试 ({attempt + 1}/{max_retries})")
            await asyncio.sleep(delay)

async def save_wechat_to_pdf(url, output_path="output.pdf", max_attempts=3):
    """终极稳定版微信文章转PDF"""
    browser = None
    context = None
    page = None
    last_error = None
    
    for attempt in range(1, max_attempts + 1):
        print(f"\n=== 第 {attempt} 次尝试 ===")
        
        try:
            async with async_playwright() as p:
                # 1. 启动浏览器（新上下文隔离）
                browser = await p.chromium.launch(
                    headless=False,
                    slow_mo=800,  # 更长的操作间隔
                    timeout=150000,
                    channel="chrome",  # 使用稳定版Chrome
                    args=[
                        f"--window-size=1240,1754",  # A4比例(1240x1754像素 @150dpi)
                        "--force-device-scale-factor=1.0"
                    ]
                )
                context = await browser.new_context(
                    user_agent="Mozilla/5.0 (iPhone; CPU iPhone OS 15_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148 MicroMessenger/8.0.28(0x18001c2f)",
                    viewport={"width": 1240, "height": 1754},  # 对应210mm×297mm @150dpi
                    #viewport={"width": 375, "height": 812},  # 移动端尺寸
                    device_scale_factor=1.0,
                )
                page = await context.new_page()

                # 2. 设置请求头
                await robust_operation(page, "set_extra_http_headers", {
                    "Referer": "https://mp.weixin.qq.com/",
                    "Accept-Language": "zh-CN,zh;q=0.9"
                })

                # 强制设置页面缩放（关键修复）
                await robust_evaluate(page, """() => {
                    const meta = document.createElement('meta');
                    meta.name = 'viewport';
                    meta.content = 'width=device-width, initial-scale=1.0, maximum-scale=1.0';
                    document.head.appendChild(meta);
                    return true;  // 明确返回结果
                }""")

                # 页面样式预处理
                await robust_operation(page, "add_init_script", """() => {
                    // 提前注入样式防止布局抖动
                    const style = document.createElement('style');
                    style.textContent = `
                        body { margin:0 !important; padding:0 !important }
                        #js_content, .rich_media_content { 
                            width:100% !important; 
                            max-width:none !important 
                        }
                    `;
                    document.head.appendChild(style);
                }""")
            
                # 3. 智能加载页面
                print("🌐 正在加载页面...")
                await robust_operation(page, "goto", url,
                    wait_until="networkidle",
                    timeout=150000,
                    referer="https://mp.weixin.qq.com/"
                )

                # 4. 反爬检测与处理
                if await robust_evaluate(page, "() => document.title.includes('验证')"):
                    print("🛡️ 检测到验证页面，尝试自动处理...")
                    # 点击操作仍需使用 robust_operation（因为不是 evaluate 操作）
                    await robust_operation(page, "click", "text=继续阅读", timeout=5000)
                    await robust_operation(page, "wait_for_load_state", "networkidle")

                # 5. 内容预处理
                print("🛠️ 异步加载图片资源...")
                await robust_evaluate(page, """async () => {
                    // 1. 处理懒加载图片（异步等待每张图加载）
                    const imgs = document.querySelectorAll('img[data-src]');
                    for (const img of imgs) {
                        img.src = img.dataset.src;
                        await new Promise((resolve) => {
                            if (img.complete) resolve();
                            else img.onload = resolve;
                        });
                    }
                    
                    // 2. 原同步操作保持不变
                    document.querySelectorAll('[style*="background-image"]').forEach(el => {
                        const bg = window.getComputedStyle(el).backgroundImage;
                        if (bg.includes('url') && !bg.includes('loaded')) {
                            el.style.backgroundImage = bg.replace('loading', 'loaded');
                        }
                    });
                    
                    // 3. 移除干扰元素（同步）
                    document.querySelectorAll('.footer, .ad-banner').forEach(el => {
                        el.remove();
                    });
    
                    // 确保正文可见
                    const content = document.getElementById('js_content');
                    if (content) content.style.visibility = 'visible';
                    
                    return true;
                }""")

                # 6. 滚动加载完整内容
                print("📜 模拟滚动加载...")
                final_height = await robust_operation(page, "evaluate", """async () => {
                    let lastHeight = 0, stableCount = 0;
                    while (stableCount < 3) {  // 连续3次高度不变才停止
                        window.scrollTo(0, document.body.scrollHeight);
                        await new Promise(r => setTimeout(r, 2000));
                        const newHeight = document.body.scrollHeight;
                        if (Math.abs(newHeight - lastHeight) < 50) stableCount++;
                        else stableCount = 0;
                        lastHeight = newHeight;
                    }
                    return lastHeight;
                }""")

                # 🔥🔥🔥 新增：移除右下角公众号水印 🔥🔥🔥
                print("🖼️ 移除右下角水印...")
                await robust_evaluate(page, """() => {
                    // 方法1：彻底删除水印容器（优先尝试）
                    const selectors = [
                        '#js_article_bottom_bar',
                        '.wx_follow_context', 
                        '.stream_bottom_bar_wrp'
                    ];
                    selectors.forEach(selector => {
                        const el = document.querySelector(selector);
                        if (el) el.style.display = 'none';
                    });
                
                    // 方法2：覆盖水印区域（备用方案）
                    const mask = document.createElement('div');
                    mask.id = '__watermark_mask__';
                    mask.style.cssText = `
                        position: fixed !important;
                        bottom: 0 !important;
                        right: 0 !important;
                        width: 150px !important;
                        height: 60px !important;
                        background: white !important;
                        z-index: 99999 !important;
                    `;
                    document.body.appendChild(mask);
                    return true;
                }""")

                await robust_evaluate(page, """() => {
                    // 精准定位水印文字父元素
                    const watermark = document.evaluate(
                        "//div[contains(@class, 'wx_follow')]//div[contains(text(), '数学张大仙')]/ancestor::div[contains(@class, 'wx_follow_')]",
                        document,
                        null,
                        XPathResult.FIRST_ORDERED_NODE_TYPE,
                        null
                    ).singleNodeValue;
                    
                    if (watermark) {
                        // 彻底移除而非隐藏
                        watermark.parentNode.removeChild(watermark);
                    } else {
                        // 暴力清除法
                        document.querySelectorAll('div').forEach(div => {
                            if (div.innerText.includes('数学张大仙')) {
                                div.style.cssText = 'display:none !important; height:0 !important;';
                            }
                        });
                    }
                    return true;
                }""")

                # 可选：验证水印是否移除（调试用）
                # await page.screenshot({ path: 'after_remove_watermark.png', fullPage: true })

                #  重置页面样式（保证内容撑满宽度）
                ret = await robust_evaluate(page, """() => {
                    document.body.style.margin = '0';
                    document.body.style.padding = '0';
                    const container = document.getElementById('js_content') || document.body;
                    container.style.width = '100%';
                    container.style.maxWidth = 'none';
                    return true;  // 明确返回执行结果
                }""")

                # 在滚动加载后添加图片加载状态检查
                loaded_status = await robust_evaluate(page, """() => {
                    const imgs = Array.from(document.querySelectorAll('img'));
                    return {
                        total: imgs.length,
                        loaded: imgs.filter(img => img.complete && img.naturalHeight > 0).length
                    };
                }""")
                print(f"📊 图片加载状态: {loaded_status['loaded']}/{loaded_status['total']}")

                # 检查页面状态
                status = await check_content_status(page)
                if status or status['loadedImgs'] < status['totalImgs'] * 0.8:
                    print(f"📊 内容状态: 高度={status['contentHeight']}px | "
                          f"图片加载={status['loadedImgs']}/{status['totalImgs']}")
                    print("⚠️ 图片加载不完全，可能影响PDF质量")
                else:
                    print("❌ 内容检查失败，继续执行...")
                
                # 强制等待剩余图片（可选）
                #await asyncio.sleep(3000)
                
                # 8. 生成PDF（带智能分页）
                print("🖨️ 正在生成A4尺寸PDF...")
                await robust_operation(page, "pdf",
                    path=output_path,
                    format="A4",
                    width="210mm",          # A4标准宽度
                    height="297mm",         # A4标准高度
                    margin={
                        "top": "15mm",
                        "right": "15mm",
                        "bottom": "15mm",
                        "left": "15mm"
                    },
                    print_background=True, 
                    scale=0.95,              # 保证边距
                    # 关键参数：设置初始视图
                    display_header_footer=False,
                    prefer_css_page_size=False,  # 关键参数
                    # 强制从第一页开始（PDF打开参数）
                    tagged=True,  # 启用文档标签
                    outline=False  # 禁用大纲视图记忆位置
                )

                print(f"✅ 成功生成PDF: {output_path}")
                return True

        except Exception as e:
            last_error = str(e)
            print(f"⚠️ 尝试 {attempt} 失败: {last_error}")
            
            # 增强错误诊断
            timestamp = int(time.time())
            try:
                if page and not page.is_closed():
                    await robust_operation(page, "screenshot", 
                        path=f"error_{timestamp}.png",
                        full_page=True,
                        quality=90
                    )
                    html = await robust_operation(page, "content")
                    with open(f"debug_{timestamp}.html", "w", encoding="utf-8") as f:
                        f.write(html)
            except Exception as debug_error:
                print(f"⚠️ 调试信息保存失败: {debug_error}")

            if attempt < max_attempts:
                print(f"🔄 {max_attempts - attempt}次重试剩余...")
                await asyncio.sleep(3)  # 重试间隔

        finally:
            # 安全释放资源
            cleanup_tasks = []
            if page and not page.is_closed():
                cleanup_tasks.append(page.close())
            if context:
                cleanup_tasks.append(context.close())
            if browser:
                cleanup_tasks.append(browser.close())
            
            if cleanup_tasks:
                await asyncio.gather(*cleanup_tasks, return_exceptions=True)

    print(f"❌ 全部{max_attempts}次尝试失败，最后错误: {last_error}")
    return False


# 调用示例
url = "https://mp.weixin.qq.com/s/kahvOoiP12OYVh-v6FNDCw"
success = await save_wechat_to_pdf(url)
print("执行结果:", "成功" if success else "失败")


=== 第 1 次尝试 ===
🌐 正在加载页面...
🛠️ 异步加载图片资源...
📜 模拟滚动加载...
🖼️ 移除右下角水印...
📊 图片加载状态: 15/20
📊 内容状态: 高度=10906px | 图片加载=15/20
⚠️ 图片加载不完全，可能影响PDF质量
🖨️ 正在生成A4尺寸PDF...
✅ 成功生成PDF: output.pdf
执行结果: 成功


In [ ]:
# 获取所有可能的水印元素信息
                watermark_info = await robust_evaluate(page, """() => {
                    const elements = document.querySelectorAll('*');
                    return Array.from(elements)
                        .filter(el => {
                            const style = window.getComputedStyle(el);
                            return (
                                style.position === 'fixed' && 
                                (style.bottom === '0px' || style.right === '0px')
                            );
                        })
                        .map(el => ({
                            html: el.outerHTML,
                            id: el.id,
                            class: el.className,
                            styles: el.getAttribute('style')
                        }));
                }""")
                print("🕵️ 检测到的可疑元素:", watermark_info)

### Compress Img

In [5]:
import tinify

def compress_with_tinify(api_key, input_path, output_path):
    tinify.key = api_key  # 替换为你的API Key
    source = tinify.from_file(input_path)
    source.to_file(output_path)

# 示例（需要先安装：pip install tinify）
compress_with_tinify("mlG0vFXH4vDh3MG14STVw0Pds8rwQLM7", "../../../Pictures/widget-pics/IMG_0123.jpeg", "../../../Pictures/widget-pics/IMG_0123_compressed.jpeg")